In [13]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import glob,os
import numpy as np
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.moo.moead import MOEAD
from pymoo.problems import *
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter
from pymoo.util.ref_dirs import get_reference_directions

In [15]:
request_count = [10,20,30]
tasks = []
for file in glob.glob(os.path.join('./pymoo/problems/nfv/sfc/data/input', '*_network.txt')):

    n = file_to_network(file)
    # requests = file_to_requests(os.path.join(args.input, f'{dat}_requests.txt'))
    for cnt in request_count:
        req_file = file.replace('network', f'{cnt}requests')
        name = req_file.split('\\')[-1][:-5]
        requests = file_to_requests(req_file)
        name = req_file.split('\\')[-1][:-5]
        tasks.append((n,requests,name))


In [16]:
rs = []
for i in range(len(tasks)):
    if i < 89:
        continue
    problem = get_problem("nfv",tasks[i][0],tasks[i][1])
    ref_dirs = get_reference_directions("uniform", 2, n_partitions=99)
    algorithm = MOEAD(
        ref_dirs,
        n_neighbors=15,
        prob_neighbor_mating=0.7,
    )
    res = minimize(problem,
                algorithm,
                ('n_gen', 50),
                seed=1,
                save_history = True,
                verbose=True)
    rs.append(res)
    X, F = res.opt.get("X", "F")
    F = np.array(F)
    np.savetxt(f'result/moead/{tasks[i][2]}.txt',F)

n_gen  |  n_eval  | n_nds  |      igd      |       gd      |       hv     
     1 |      100 |      3 |  0.5632984919 |  0.1126671935 |  0.1996666667
     2 |      200 |     16 |  0.5632984919 |  0.1256476822 |  0.1996666667
     3 |      300 |     52 |  0.5632984919 |  0.1065225835 |  0.1996666667
     4 |      400 |     77 |  0.5632984919 |  0.1018782159 |  0.1996666667
     5 |      500 |     79 |  0.5632984919 |  0.1013424439 |  0.1996666667
     6 |      600 |     79 |  0.5632984919 |  0.1013424439 |  0.1996666667
     7 |      700 |     79 |  0.5632984919 |  0.1013424439 |  0.1996666667
     8 |      800 |     79 |  0.5632984919 |  0.1013424439 |  0.1996666667
     9 |      900 |     96 |  0.5632984919 |  0.0976897065 |  0.1996666667
    10 |     1000 |     96 |  0.5632984919 |  0.0976897065 |  0.1996666667
    11 |     1100 |     35 |  0.5632280660 |  0.1272614257 |  0.2000000000
    12 |     1200 |     68 |  0.5616193809 |  0.0960013967 |  0.2050000000
    13 |     1300 |     8